# Projections

Xvec supports handling Coordinate Reference Systems (CRS) using the PyPROJ package. Each GeometryIndex has a `.crs` attribute storing the CRS as a `pyproj.CRS` object (similarly to GeoPandas), and the same object is also stored in the `attrs` of the respective DataArray.

In [2]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import xvec

You can create the same Dataset of NYC taxi trips as in the [previous notebook](indexing), just backed by a sparse array to save memory this time.

In [3]:
# Load the data
trips = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet"
)  # 33MB
zones = gpd.read_file(
    "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip"
)  # 1MB
lookup = pd.read_csv("https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv")

# create variables for day and hour
trips["date"] = trips.tpep_pickup_datetime.dt.date
trips["hour"] = trips.tpep_pickup_datetime.dt.hour

# use groupby over five columns to create a mutli-indexed DataFrame with aggregations
# and create a Dataset backed by sparse arrays
taxi_trips = xr.Dataset.from_dataframe(
    trips[  # filter only trips with known locations
        trips.PULocationID.isin(zones.LocationID)
        & trips.DOLocationID.isin(zones.LocationID)
    ]
    .groupby(["payment_type", "date", "hour", "PULocationID", "DOLocationID"])
    .agg(
        {
            "trip_distance": "mean",
            "VendorID": "count",
            "tip_amount": "mean",
            "fare_amount": "mean",
        }
    ),
    sparse=True,
)

# Replace int codes with labels
taxi_trips["payment_type"] = [
    "Credit card",
    "Cash",
    "No charge",
    "Dispute",
    "Unknown",
    "Voided trip",
]

# create linkable geometry variable
taxi_zones = (
    lookup.merge(
        zones.dissolve("LocationID")[["zone", "geometry"]],
        left_on="Zone",
        right_on="zone",
        how="left",
    )
    .set_index("LocationID")
    .geometry
)
# replace location IDs with actual geometries
taxi_trips["PULocationID"] = taxi_zones.loc[taxi_trips.PULocationID].values
taxi_trips["DOLocationID"] = taxi_zones.loc[taxi_trips.DOLocationID].values

# rename
taxi_trips = taxi_trips.rename(
    {"PULocationID": "origin", "DOLocationID": "destination", "VendorID": "trips_count"}
)

# assing GeometryIndex
taxi_trips = taxi_trips.xvec.set_geom_indexes(["origin", "destination"], crs=zones.crs)
taxi_trips

Format,coo
Data Type,float64
Shape,"(6, 40, 24, 252, 257)"
nnz,1268780
Density,0.003401184385701247
Read-only,True
Size,21.8M
Storage ratio,0.0
Format,coo
Data Type,float64
Shape,"(6, 40, 24, 252, 257)"


You can see that each `GeometryIndex` has a CRS assigned.

In [4]:
taxi_trips.xindexes

Indexes:
    payment_type  PandasIndex
    date          PandasIndex
    hour          PandasIndex
    origin        GeometryIndex (crs=EPSG:2263)
    destination   GeometryIndex (crs=EPSG:2263)

The `pyproj.CRS` class is available via the `.crs` attribute.

In [5]:
taxi_trips.origin.crs

<Derived Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

And in the `attrs`. This duplication ensures that the CRS information is preserved when the dimension loses `GeometryIndex` due to some operations.

In [6]:
taxi_trips.origin.attrs["crs"]

<Derived Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

## Setting a Projection

Like in GeoPandas, there are two relevant operations for projections: setting a projection and re-projecting.

Unlike GeoPandas, the vector data cube is more often created in memory from other objects than read from a file, so a user is responsible for setting a projection to every DataArray with geometries.

The most common way to set a projection is to create a `GeometryIndex` using the `.xvec.set_geom_indexes()` method. It takes a `crs` attribute that assigns the CRS to all indexes created. Above, the CRS is assigned on this line, taking the CRS information directly from the original GeoDataFrame:

```py
taxi_trips.xvec.set_geom_indexes(["origin", "destination"], crs=zones.crs)
```

If you don't assign a CRS at this stage or each of your `GeometryIndex` uses a different one, you can use the `.xvec.set_crs()` method later. However, the method does not allow you to override existing CRS that is different unless you specify `allow_override=True`. Note that you should rarely need to override a CRS.[^override]

[^override]: {-} Note that assignment of a CRS via `.xvec.set_crs()` does **NOT** change the underlying geometry. If you want to re-project to a different CRS, use `.xvec.to_crs()` instead.

In [8]:
taxi_trips.xvec.set_crs(origin=4326, destination=3857, allow_override=True)

Format,coo
Data Type,float64
Shape,"(6, 40, 24, 252, 257)"
nnz,1268780
Density,0.003401184385701247
Read-only,True
Size,21.8M
Storage ratio,0.0
Format,coo
Data Type,float64
Shape,"(6, 40, 24, 252, 257)"


## Re-projecting

Geometries can be re-projected from one CRS to another using the `.xvec.to_crs()` method. You can see that not only the CRS information has changed but also the coordinates of geometries.

In [9]:
taxi_trips.xvec.to_crs(origin=4326, destination=3857)

Format,coo
Data Type,float64
Shape,"(6, 40, 24, 252, 257)"
nnz,1268780
Density,0.003401184385701247
Read-only,True
Size,21.8M
Storage ratio,0.0
Format,coo
Data Type,float64
Shape,"(6, 40, 24, 252, 257)"


## CRS input

You can pass CRS information in nearly any common format. The `pyproj.CRS` class is created via the `pyproj.CRS.from_user_input()` method that can take anything like:

- PROJ string
- Dictionary of PROJ parameters
- PROJ keyword arguments for parameters
- JSON string with PROJ parameters
- CRS WKT string
- An authority string [i.e. `"epsg:4326"`]
- An EPSG integer code [i.e. `4326`]
- A tuple of ("auth_name": "auth_code") [i.e `("epsg", "4326")`]
- An object with a `to_wkt` method.
- A `pyproj.crs.CRS` class

When passing a CRS from a GeoPandas object, you pass a `pyproj.crs.CRS` class.